In [24]:
import os
import subprocess
from bs4 import BeautifulSoup

In [25]:
path_default = "data/"
path_target = "articles.A-B.xml"
path_opennlp = "apache-opennlp-1.8.1/"

In [26]:
labels = {}
file_label = open(path_default + 'Structured-Abstracts-Labels-102615.txt', 'r')
for line in file_label.readlines() :
    label = line.split('|')
    labels[label[0]] = label[1]
    
#labels

In [27]:
comment = 'Two symptom-based screening questionnaires, the Berlin Questionnaire (BQ) and the Epworth Sleepiness Scale (ESS), were administered to enroll 1509 gravidae. Screen positive subjects were referred for polysomnography (PSG). The primary outcome was the occurrence of either gestational hypertension or preeclampsia. [a1]Generalized linear models (GLM) were used to estimate the relative risks of associations.'
process_nlp = subprocess.Popen([path_opennlp + 'bin/opennlp',  'SentenceDetector', path_opennlp + 'en-sent.bin'], stdin = subprocess.PIPE, stdout = subprocess.PIPE, stderr = subprocess.PIPE)
result, stderr = process_nlp.communicate(comment.encode())
for line in result.decode().split('\n') :
    print(line)

Two symptom-based screening questionnaires, the Berlin Questionnaire (BQ) and the Epworth Sleepiness Scale (ESS), were administered to enroll 1509 gravidae.
Screen positive subjects were referred for polysomnography (PSG).
The primary outcome was the occurrence of either gestational hypertension or preeclampsia.
[a1]Generalized linear models (GLM) were used to estimate the relative risks of associations.

Execution time: 0.086 seconds



In [30]:
lines = list(map(lambda s : s.strip(), open('/home/kmh/Dataset/' + path_target + '.txt', 'r').readlines()))

i = 0
for path_file in lines :
    if i == 2 :
        break
    
    article = BeautifulSoup(open('/home/kmh/Dataset/' + path_target + '/' + path_file, 'r').read(), 'html.parser')
    
    for node in article.select('abstract sec') :
            print('Original Label: ' + node.title.text.upper())
            print('Mapping Label: ' + labels[node.title.text.upper()])
            print(node.p.text + '\n')

    i += 1

Original Label: OBJECTIVE
Mapping Label: OBJECTIVE
This prospective observational study explored the association of hypertensive disorders of pregnancy and small-for-gestational age (SGA) with obstructive sleep apnea (OSA) as determined by screening measures for OSA and sleep studies.

Original Label: STUDY DESIGN
Mapping Label: METHODS
Two symptom-based screening questionnaires, the Berlin Questionnaire (BQ) and the Epworth Sleepiness Scale (ESS), were administered to enroll 1509 gravidae. Screen positive subjects were referred for polysomnography (PSG). The primary outcome was the occurrence of either gestational hypertension or preeclampsia. [a1]Generalized linear models (GLM) were used to estimate the relative risks of associations.

Original Label: RESULTS
Mapping Label: RESULTS
1157 subjects were available for outcomes analysis. Screening positive on the BQ was positively associated with hypertensive disorders in GLM models (aRR=1.90, 95%CI 1.52–2.37).

Original Label: CONCLUSION